In [1]:
from collections import Counter

import pandas as pd
import numpy as np

from lol_fandom import SITE
from lol_fandom import get_leagues, get_tournaments
from lol_fandom import get_scoreboard_games, get_scoreboard_players
from lol_fandom import from_response

pd.set_option('display.max_columns', None)

In [2]:
TARGET_LEAGUES = [
    'LTC', 'LCK',
    'LPL',
    'EU LCS', 'LEC',
    'NA LCS', 'LCS',
    'GPL', 'LST',
    'LLA', 'CLS', 'LLN',
    'LCO', 'OPL',
    'LCL',
    'LJL',
    'LMS', 'PCS',
    'VCS',
    'CBLOL', 
    'TCL',
    
    'MSI',
    'WCS',
]

In [3]:
def get_new_id(ids):
    if 'team_id' in ids.columns:
        id_list = sorted(ids['team_id'].unique())
    else:
        id_list = sorted(ids['player_id'].unique())

    if len(id_list) == 0:
        return 1
    prev = id_list[0]
    for id in id_list[1:]:
        if prev + 1 != id:
            return prev + 1
        prev = id
    return prev + 1

## Teams

In [4]:
leagues = get_leagues(where='L.Level="Primary" and L.IsOfficial="Yes"')
leagues

,League,League Short,Region,Level,IsOfficial
0,2015 International Wildcard Tournament,2015 IWCT,International,Primary,Yes
1,2016 International Wildcard Qualifier,IWCQ,International,Primary,Yes
2,Circuit Brazilian League of Legends,CBLOL,Brazil,Primary,Yes
3,Copa Latinoamérica Sur,CLS,LAS,Primary,Yes
4,DreamHack,DH,International,Primary,Yes
5,Europe League Championship Series,EU LCS,Europe,Primary,Yes
6,Garena Premier League,GPL,SEA,Primary,Yes
7,International Wildcard Invitational,2015 IWCI,International,Primary,Yes
8,League of Legends Championship Series,LCS,North America,Primary,Yes
9,League of Legends SEA Tour,LST,SEA,Primary,Yes


In [5]:
lst = []
for league in TARGET_LEAGUES:
    if league not in leagues['League Short'].values:
        lst.append(league)
lst

[]

In [9]:
teams = pd.read_csv('./csv/teams_id.csv')
names = []

# for year in range(2011, 2024):
for year in range(2023, 2024):
    print(year)
    for league in leagues['League Short']:
        print(f'\t{league}')
        tournaments = get_tournaments(
            where=f'L.League_Short="{league}" and T.Year={year}'
        )
        if tournaments.shape[0] == 0:
            continue
        tournaments = tournaments.sort_values(
            by=['Year', 'DateStart', 'Date']
        ).reset_index(drop=True)

        for page in tournaments['OverviewPage']:
            print(f'\t\t{page}')
            try:
                scoreboard_games = get_scoreboard_games(
                    where=f'T.OverviewPage="{page}"'
                )
                if scoreboard_games is None:
                    continue
            except Exception as e:
                print(e)
                continue
            scoreboard_games = scoreboard_games.sort_values(
                by='DateTime UTC'
            ).reset_index(drop=True)
            team_names = scoreboard_games[['Team1', 'Team2']].unstack().unique()
            names = []
            for name in team_names:
                if name not in teams['team'].values:
                    names.append(name)
            if len(names) > 0:
                names = sorted(names)
                break
        if len(names) > 0:
            break
    if len(names) > 0:
        print(scoreboard_games['OverviewPage'].unique()[0])
        print(names)
        break

2023
	2015 IWCT
	IWCQ
	CBLOL
		CBLOL/2023 Season/Split 1
	CLS
	DH
	EU LCS
	GPL
	2015 IWCI
	LCS
		LCS/2023 Season/Spring Season
	LST
	LLA
		LLA/2023 Season/Opening Promotion
		LLA/2023 Season/Opening Season
		LLA/2023 Season/Opening Playoffs
	LLN
	LCK
		LCK/2023 Season/Spring Season
		LCK/2023 Season/Spring Playoffs
	LCO
		LCO/2023 Season/Split 1 Stage 1
		LCO/2023 Season/Split 1 Stage 2
	LCL
	LEC
		LEC/2023 Season/Winter Season
		LEC/2023 Season/Winter Groups
		LEC/2023 Season/Winter Playoffs
		LEC/2023 Season/Season Finals
	LJL
		LJL/2023 Season/Spring Season
		LJL/2023 Season/Spring Playoffs
	LMS
	LTC
	MLG
	MSC
	MSI
		2023 Mid-Season Invitational
	NA LCS
	OPL
	PCS
		PCS/2023 Season/Spring Season
		PCS/2023 Season/Spring Playoffs
	RR
	RR
	RR
	RR
	RR
	Riot
	LPL
		LPL/2023 Season/Spring Season
		LPL/2023 Season/Spring Playoffs
	TCL
		TCL/2023 Season/Winter Season
	VCS
		VCS/2023 Season/Spring Season
	WCS
		LCK/2023 Season/Regional Finals


In [ ]:
teams.loc[teams['team'].str.contains('g2', case=False)]

In [ ]:
if len(names) > 0:
    teams = pd.concat(
        [
            teams,
            pd.Series({'team': names.pop(0), 'team_id': get_new_id(teams)}).to_frame().T
            # pd.Series({'team': names.pop(0), 'team_id': 259}).to_frame().T
        ],
        ignore_index=True
    )
names

In [ ]:
teams = teams.sort_values(by='team').reset_index(drop=True)
teams

In [ ]:
teams.to_csv('./csv/teams_id.csv', index=False)

## Players

In [1852]:
players = pd.read_csv('./csv/players_id.csv')
names = {}

# for year in range(2011, 2024):
for year in range(2015, 2024):
    print(year)
    for league in leagues['League Short']:
        print(f'\t{league}')
        tournaments = get_tournaments(
            where=f'L.League_Short="{league}" and T.Year={year}'
        )
        if tournaments.shape[0] == 0:
            continue
        tournaments = tournaments.sort_values(
            by=['Year', 'DateStart', 'Date']
        ).reset_index(drop=True)

        for page in tournaments['OverviewPage']:
            print(f'\t\t{page}')
            try:
                scoreboard_games = get_scoreboard_games(
                    where=f'T.OverviewPage="{page}"'
                )
                if scoreboard_games is None:
                    continue
            except Exception as e:
                print(e)
                continue
            lst_team = scoreboard_games[['Team1', 'Team2']].unstack().unique()
            player_names = []
            for team in lst_team:
                print(f'\t\t\t{team}')
                try:
                    scoreboard_players = get_scoreboard_players(
                        where=f'T.OverviewPage="{page}" and SP.Team="{team}"'
                    )
                    if scoreboard_players is None:
                        continue
                except Exception as e:
                    print(e)
                    continue
                player_names += list(set(
                    scoreboard_players[['Link', 'Team']].itertuples(index=False)
                ))
            names = {}
            for (name, team) in player_names:
                if name not in players['player'].values:
                    lst = names.get(team, [])
                    lst.append(name)
                    names[team] = lst
            if len(names) > 0:
                break
        if len(names) > 0:
            break
    if len(names) > 0:
        print(scoreboard_players['OverviewPage'].unique()[0])
        for team, player_lst in names.items():
            print(f'{team}\n  {player_lst}')
        break

2015
	2015 IWCT
	IWCQ
	CBLOL
		CBLOL/2015 Season/Split 1 Qualifiers
		CBLOL/2015 Season/Split 1
			CNB e-Sports Club
			Dexterity Team
			INTZ
			JAYOB e-Sports
			KaBuM! Black
			KaBuM! Orange
			Keyd Stars
			paiN Gaming
		CBLOL/2015 Season/Split 1 Playoffs
			CNB e-Sports Club
			Dexterity Team
			INTZ
			KaBuM! Black
			Keyd Stars
			paiN Gaming
		CBLOL/2015 Season/Split 2 Promotion
			Dexterity Team
			IMP e-Sports
			INTZ Red
			JAYOB e-Sports
			KaBuM! Orange
			Keyd Warriors
		CBLOL/2015 Season/Split 2
			CNB e-Sports Club
			g3nerationX
			INTZ
			INTZ Red
			KaBuM! Black
			KaBuM! Orange
			Keyd Stars
			paiN Gaming
		CBLOL/2015 Season/Split 2 Playoffs
			CNB e-Sports Club
			g3nerationX
			INTZ
			INTZ Red
			Keyd Stars
			paiN Gaming
		CBLOL/2015 Season/Post-Season
			Big Gods
			CNB e-Sports Club
			g3nerationX
			INTZ
			INTZ Red
			KaBuM! Black
			KaBuM! Orange
			Keyd Stars
	CLS
		Latin America Cup 2015/LAS/Opening Cup/Promotion
		Latin America Cup 2015/LAS/Opening Cup/

In [1853]:
print(scoreboard_players['OverviewPage'].unique()[0])
teams = list(names.keys())
print(teams)
print()
print(teams[0])
print(names[teams[0]])

LJL/2015 Season/Season 1
['7th heaven', 'DetonatioN RabbitFive', 'Ozone Rampage', 'Rascal Jester', 'Salvage Javelin']

7th heaven
['Mueki', 'Clockday', 'enty', 'yunn', 'ThintoN', 'Razer', 'yoshiaki3110']


In [1923]:
players.loc[players['player'].str.contains(
    'raphon',
    case=False
)]

,player,player_id


In [1925]:
if len(names) > 0:
    name = names[teams[0]].pop(0)
    players = pd.concat(
        [
            players,
            pd.Series({'player': name, 'player_id': get_new_id(players)}).to_frame().T
            # pd.Series({'player': name, 'player_id': 15}).to_frame().T
        ],
        ignore_index=True
    )
    if len(names[teams[0]]) == 0:
        del names[teams[0]]
        del teams[0]
print(teams)
print()
if len(teams) > 0:
    print(teams[0])
    print(names[teams[0]])

[]



In [1926]:
players = players.sort_values(by=['player_id', 'player']).reset_index(drop=True)
players

,player,player_id
0,Bear (Manuel Parrochia),1
1,BearJew,1
2,Plugo,2
3,Element,3
4,element,3
...,...,...
2139,Cool (Ryohei Tsuji),1777
2140,Joke,1778
2141,toshibu,1779
2142,Sweet (Ryota Murakami),1780


In [1927]:
while True:
    players['lower'] = players['player'].str.lower()
    lst = sorted(list(players[['lower', 'player_id']].itertuples(index=False)))

    is_complete = True
    prev = lst[0]
    for cur in lst[1:]:
        if prev[0] == cur[0] and prev[1] != cur[1]:
            is_complete = False
            break
        prev = cur

    if not is_complete:
        print('incomplete')
        candidate = players.loc[players['lower'] == prev[0]]
        print(candidate)
        df = players.loc[players['player_id'].isin(candidate['player_id'].values)]
        if candidate.shape[0] != df.shape[0]:
            print('improper id')
            print(df)
            break
        else:
            players.loc[players['lower'] == prev[0], 'player_id'] = min(players.loc[players['lower'] == prev[0], 'player_id'])
            print('after correction')
            print(players.loc[players['lower'] == prev[0]])
            print('\n')
    else:
        print('completed')
        break

completed


In [1928]:
is_complete = True

counter = Counter(players[['player', 'player_id']].itertuples(index=False))
lst = [key for key, value in counter.items() if value > 1]
if len(lst) > 0:
    is_complete = False
    print('incomplete')
    print(lst)
    for (name, id) in lst:
        idx = players.loc[players['player'] == name].index
        players.drop(idx[1:], inplace=True)
else:
    print('completed')

completed


In [1929]:
if is_complete:
    players[['player', 'player_id']].to_csv('./csv/players_id.csv', index=False)
    print('save complete')

save complete


## Scoreboard

In [ ]:
leagues = get_leagues(where='L.Level="Primary" and L.IsOfficial="Yes"')
leagues

In [ ]:
# for year in range(2011, 2024):
for year in range(2023, 2024):
    tournaments = pd.DataFrame()
    # for league in TARGET_LEAGUES:
    for league in leagues['League Short']:
        t = get_tournaments(where=f'L.League_Short="{league}" and T.Year={year}')
        tournaments = pd.concat([tournaments, t])
    tournaments = tournaments.sort_values(
        by=['Year', 'DateStart', 'Date']
    ).reset_index(drop=True)

    tournaments.to_csv(f'./csv/tournaments/{year}_tournaments.csv', index=False)
    print(year, tournaments.shape)

In [ ]:
# for year in range(2011, 2024):
for year in range(2023, 2024):
    tournaments = pd.read_csv(f'./csv/tournaments/{year}_tournaments.csv')
    print(year, tournaments.shape)
    scoreboard_games = pd.DataFrame()
    for page in tournaments['OverviewPage']:
        sg = get_scoreboard_games(where=f'T.OverviewPage="{page}"')
        if sg is None:
            print(f'{year}\t{page:35}\tdrop')
            tournaments.drop(tournaments.loc[tournaments['OverviewPage'] == page].index, inplace=True)
            continue
        league = tournaments.loc[tournaments['OverviewPage'] == page, 'League'].iloc[0]
        league = leagues.loc[leagues['League'] == league, 'League Short'].iloc[0]
        sg['League'] = league
        sg = sg.sort_values(
            by='DateTime UTC'
        ).reset_index(drop=True)
        print(f'\t{sg.shape}\t{page}')
        scoreboard_games = pd.concat([scoreboard_games, sg])
    scoreboard_games.to_csv(f'./csv/scoreboard_games/{year}_scoreboard_games.csv', index=False)
    print(f'{year} scoreboard_games {scoreboard_games.shape}')
    tournaments.to_csv(f'./csv/tournaments/{year}_tournaments.csv', index=False)
    print(f'{year} tournaments {tournaments.shape}')

In [ ]:
scoreboard_games